In [17]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split,KFold,cross_validate
import sklearn.datasets
import optuna 

In [20]:
def objectives(trial):
    # scikit-learnでお試しデータの準備
    boston = sklearn.datasets.load_boston()
    X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, random_state=0)

    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
        #fixed
        'boosting_type':'gbdt',
        'max_depth':-1,
        'learning_rate':0.1,
        'n_estimators': 1000,
        'metric':'l2',

        #variable
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'reg_alpha': trial.suggest_loguniform('reg_alpha',0.001, 10),
        'reg_lambda':trial.suggest_loguniform('reg_lambda', 0.001, 10),

    }

    # LightGBMで学習+予測
    model = lgb.LGBMRegressor(**params,random_state=0)# 追加部分
    
    # kFold交差検定で決定係数を算出し、各セットの平均値を返す
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    scores = cross_validate(model, X=X_train, y=y_train,scoring='r2',cv=kf)   

    return scores['test_score'].mean()

In [23]:
%%time

# optunaによる最適化呼び出し
opt = optuna.create_study(direction='maximize')
opt.optimize(objectives, n_trials=5)


# 実行結果表示
print('最終トライアル回数: {}'.format(len(opt.trials)))
print('ベストトライアル:')
trial = opt.best_trial
print('値: {}'.format(trial.value))
print('パラメータ:')
for key, value in trial.params.items():
    print('{}: {}'.format(key, value))

[I 2019-11-17 21:26:11,525] Finished trial#0 resulted in value: 0.8777692421511626. Current best value is 0.8777692421511626 with parameters: {'num_leaves': 48, 'reg_alpha': 0.02841795034878006, 'reg_lambda': 0.02699956507604085}.
[I 2019-11-17 21:26:12,234] Finished trial#1 resulted in value: 0.8772753774771155. Current best value is 0.8777692421511626 with parameters: {'num_leaves': 48, 'reg_alpha': 0.02841795034878006, 'reg_lambda': 0.02699956507604085}.
[I 2019-11-17 21:26:12,892] Finished trial#2 resulted in value: 0.8781465260185032. Current best value is 0.8781465260185032 with parameters: {'num_leaves': 38, 'reg_alpha': 0.2613680772014017, 'reg_lambda': 0.0349231330790776}.
[I 2019-11-17 21:26:13,231] Finished trial#3 resulted in value: 0.8735889819375333. Current best value is 0.8781465260185032 with parameters: {'num_leaves': 38, 'reg_alpha': 0.2613680772014017, 'reg_lambda': 0.0349231330790776}.
[I 2019-11-17 21:26:13,526] Finished trial#4 resulted in value: 0.87431469403108

最終トライアル回数: 5
ベストトライアル:
値: 0.8781465260185032
パラメータ:
num_leaves: 38
reg_alpha: 0.2613680772014017
reg_lambda: 0.0349231330790776
Wall time: 2.77 s
